<a href="https://www.kaggle.com/code/aathirasanthosh/analyzing-american-baby-name-trends-using-sql?scriptVersionId=120649986" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Analyzing American Baby Name Trends Using SQL**
![hello_my_name_is_cards](https://upload.wikimedia.org/wikipedia/commons/2/26/Hello_My_Name_Is_%2815283079263%29.jpg?20160131002645)

This project uses data provided by the United States Social Security Administration from 1880-2014 to investigate how American baby names have changed over the years. We will be using SQL to analyze what names have stood the test of time.

In [1]:
#Connecting to SQL Database
import sqlite3
import pandas as pd

conn= sqlite3.connect("datacamp_python.db")

In [2]:
#Importing CSV file
names = pd.read_csv("/kaggle/input/us-baby-names/NationalNames.csv")
names.to_sql(
            'baby_names',             # Name of the sql table
             conn,                 # sqlite.Connection or sqlalchemy.engine.Engine
             if_exists='replace'
            )

Let's first take a look at what names have appeared in all years of our study period.

In [3]:
cursor = conn.cursor()
data_sql_1 = pd.read_sql("""
                        SELECT Name, 
                        SUM(Count) AS Sum 
                        FROM baby_names 
                        GROUP BY Name
                        HAVING COUNT(*) = 135
                        ORDER BY sum DESC
                        ;
                        """,
                        conn)
print(data_sql_1)



           Name    Sum
0    Alexandria  93397
1     Madeleine  36645
2        Helena  33074
3        Helene  32942
4       Mariana  32820
..          ...    ...
137     Sanders   2227
138       Macon   1907
139      Little   1781
140      Mackie   1556
141     Ventura   1467

[142 rows x 2 columns]


Wow! It looks like Alexandria is a really popular female name that has stood the test of time. In fact, the Top 5 popular names are all female.

Let's broaden our understanding of the dataset by looking at all names. We shall attempt to categorize each name into a popularity type (Classic, Semi-Classic, Semi-trendy and Trendy) based on how often they show up in our dataset. 

In [4]:
data_sql_2 = pd.read_sql("""
                        SELECT Name,
                        SUM(Count) AS Sum,
                        CASE WHEN COUNT(*) > 80 THEN 'Classic'
                         WHEN COUNT(*) > 50 THEN 'Semi-classic'
                         WHEN COUNT(*) > 20 THEN 'Semi-trendy'
                        ELSE 'Trendy' END AS Popularity_Type
                        FROM baby_names
                        GROUP BY Name
                        ORDER BY Name
                        ;
                        """,
                        conn)
print(data_sql_2)

            Name  Sum Popularity_Type
0          Aaban   72          Trendy
1          Aabha   21          Trendy
2          Aabid    5          Trendy
3      Aabriella   10          Trendy
4          Aadam  196     Semi-trendy
...          ...  ...             ...
93884  Zytavious   43          Trendy
93885     Zyvion    5          Trendy
93886    Zyyanna    6          Trendy
93887      Zyyon    6          Trendy
93888      Zzyzx    5          Trendy

[93889 rows x 3 columns]


Since we didn't really see a lot of traditional male names on the top for our first SQL query, let's filter to only the male names in our dataset and rank them based on the number of babies that have ever been given that name.

In [5]:
data_sql_3 = pd.read_sql("""
                        SELECT Name,
                        SUM(Count) AS Sum,
                        RANK() OVER (ORDER BY SUM(Count) DESC) AS Name_Rank
                        FROM baby_names 
                        WHERE Gender = 'M'
                        GROUP BY Name
                        LIMIT 10
                        ;
                        """,
                        conn)
print(data_sql_3)

      Name      Sum  Name_Rank
0    James  5105919          1
1     John  5084943          2
2   Robert  4796695          3
3  Michael  4309198          4
4  William  4055473          5
5    David  3577704          6
6   Joseph  2570095          7
7  Richard  2555330          8
8  Charles  2364332          9
9   Thomas  2283080         10


Looks like James and John were really popular names!

Next, let's take a look at the Top 10 popular female names of 2014 that ended in the letter A.

In [6]:
data_sql_4 = pd.read_sql("""
                        SELECT Name
                        FROM baby_names
                        WHERE Gender = 'F' AND Year = 2014 AND Name LIKE '%a'
                        GROUP BY Name
                        ORDER BY SUM(Count) DESC
                        LIMIT 10
                        ;
                        """,
                        conn)
print(data_sql_4)

       Name
0      Emma
1    Olivia
2    Sophia
3  Isabella
4       Ava
5       Mia
6     Sofia
7    Amelia
8      Ella
9  Victoria


Let's now explore the top male names in each year.

In [7]:
data_sql_6 = pd.read_sql("""
                        SELECT B.Year,
                        A.Name,
                        B.Count
                        FROM baby_names AS A
                        right join (SELECT Year,
                        MAX(Count) AS Count
                        FROM baby_names
                        WHERE Gender = 'M'
                        GROUP BY Year) AS B
                        ON A.Year = B.Year AND A.Count = B.Count
                        ORDER BY B.Year DESC
                        ;
                        """,
                        conn)
print(data_sql_6)

     Year   Name  Count
0    2014   Noah  19144
1    2013   Noah  18179
2    2012  Jacob  19030
3    2011  Jacob  20331
4    2010  Jacob  22082
..    ...    ...    ...
130  1884   John   9388
131  1883   John   8894
132  1882   John   9557
133  1881   John   8769
134  1880   John   9655

[135 rows x 3 columns]


Looks like Noah and Jacob were the top names in 2010-2014. However, if we look at the early 1800 years, John also appears to have been a popular name. 

Now let's see which male name has been number one for the largest number of years

In [8]:
data_sql_7 = pd.read_sql("""
                        WITH TEMP AS (SELECT B.Year,
                        A.Name,
                        B.Max_Sum
                        FROM baby_names AS A
                        right join (SELECT Year,
                        MAX(Count) AS Max_Sum
                        FROM baby_names
                        WHERE Gender = 'M'
                        GROUP BY Year) AS B
                        ON A.Year = B.Year AND A.Count = B.Max_Sum
                        ORDER BY B.Year DESC)

                        SELECT Name,
                        COUNT(*) AS Count_Top_Name
                        FROM TEMP
                        GROUP BY Name
                        ORDER BY Count_Top_Name DESC
                        ;
                        """,
                        conn)
print(data_sql_7)

      Name  Count_Top_Name
0  Michael              44
1     John              44
2   Robert              17
3    Jacob              14
4    James              13
5     Noah               2
6    David               1


Looks like it's a tie between Michael and John, both appearing in the top spot for the largest number of years between 1880-2014.